# %matplotlib notebook
import pandas as pd
entrada = '/home/fernando/Git/corpus/Corpus/romances.csv'
entrada = '/home/fernando/Git/corpus/Corpus/ArticuloDivinoJason/romances/cabecera.csv'

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Algunas variables para no truncar los df al imprimirlos
# No correr si no se necesita examinar línea a línea

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', None)

In [ ]:
# Cargo los datos de entrada
entrada = 'romances.csv'
df = pd.read_csv(entrada)
#dftest = pd.read_csv(entrenamiento)
dfcontrol =  df.loc[df['Autor'].isin([ 'C01', 'C02', 'C03', 'C04',
                                     'M01', 'M02', 'M03', 'M04'])]
dfdudosos =  df.loc[df['Autor'].isin([ 'Jasón', 'Redención'])]
#df = dfdatos.append(dftest, ignore_index=True)

#df.append(dftest).append(control)
# Los ritmos con + y - dan problemas como nombres de columna. Reemplazar los caracteres problemáticos lo soluciona
df['Ritmo'] = df['Ritmo'].str.replace('+','X').str.replace('-', 'o')
#Primera prueba (Cambioar para la verificación)
#dfcontrol = df[df['Autor'].isin(['Calderón', 'Mira', 'Lope', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06',
#                                'C07', 'C08', 'C09', 'C10', 'C11', 'C12',
#                                 'M01', 'M02', 'M03', 'M04', 'M05', 'M06'])]
#df = df[df['Autor'].isin(['Calderón', 'Mira', 'Lope', 'X', 'Y'])]

In [ ]:
#Defino variab, ignore_index=Trueles globales
# Seis obras atribuidas a Mira y 12 autógrafos de Calderón
control = ['C01', 'C02', 'C03', 'C04', 'M01', 'M02', 'M03', 'M04']
candidatos = ['Calderón', 'Lope', 'Mira']
dudosos = ['Jasón', 'Redención']

In [ ]:
# Averigua el número de muestras de cada obra y devuelve un df
# con un número de muestras homogéneo para todas las obras
def muestrasiguales(data, i=0):
    resultado = pd.DataFrame()
    obras = pd.unique(data['Obra'])
    if i == 0:
        menor = min([len(data[data['Obra'] == obra]) for obra in obras])
    else:
        menor = 100
    for obra in obras:
        resultado = resultado.append(
# Usamos 42 como valor de aleatorización por tratarse de la respuesta a la vida, el universo y todo (cf. Adams 1979) 
            data[data['Obra'] == obra].sample(n=menor, random_state = 42), ignore_index=True
        )
    return (resultado, menor)

In [ ]:
# Esta función añade columna con el recuento de ritmos de un tipo por obra y recuento relativo versos/total
def recuento(data, columna):
    dfritmo = pd.DataFrame()
    valores = pd.unique(data[columna])
    obras = pd.unique(data['Obra'])
    for obra in obras:
        nversos = len(data[data['Obra'] == obra])
        cuentas = {}
        autor =  data.loc[data['Obra'] == obra]['Autor'].value_counts()[:1].index.tolist()[0]
        for valor in valores:
            fila = {'Obra': obra, 'Autor': autor}
            cuenta = len(data.loc[data['Obra'] == obra][data[columna] == valor])
            relcuenta = cuenta/nversos
            fila = {'Obra': obra, 'Autor': autor, columna: valor, 'Cuenta': cuenta, 'RelCuenta' : relcuenta}
            if cuenta > 0:           
                dfritmo = dfritmo.append(fila, ignore_index=True)
    return dfritmo.convert_dtypes()

In [ ]:
def recuento(data, columna):
    dfritmo = pd.DataFrame()
    valores = pd.unique(data[columna])
    obras = pd.unique(data['Obra'])
    for obra in obras:
        nversos = len(data[data['Obra'] == obra])
        cuentas = {}
        autor =  data.loc[data['Obra'] == obra]['Autor'].value_counts()[:1].index.tolist()[0]
        for valor in valores:
            fila = {'Obra': obra, 'Autor': autor}
            cuenta = len(data.loc[data['Obra'] == obra][data[columna] == valor])
            relcuenta = cuenta/nversos
            fila = {'Obra': obra, 'Autor': autor, columna: valor, 'Cuenta': cuenta, 'RelCuenta' : relcuenta}
            if cuenta > 0:           
                dfritmo = dfritmo.append(fila, ignore_index=True)
    return dfritmo.convert_dtypes()

In [ ]:
# Esta función pasa los valores de ritmo a variables para formar una tabla en formato largo, que 
# se puede usar con MANOVA.wide de R o con las pruebas de scikitlearning

def formatolargo(columna, data, cuenta='Cuenta'):
    dflargo = pd.DataFrame()
    obras = pd.unique(data['Obra'])
    variables = pd.unique(data[columna])
    calderon = lope = mira = 0
    for obra in obras:
        subconjunto = data[data['Obra'] == obra]
        autor = max(subconjunto['Autor'])
        fila = {'Autor': autor, 'Obra': obra}
        for var in variables:
            suma = subconjunto[subconjunto[columna] == var]['Cuenta'].sum()
            fila.update({var: suma})
        dflargo = dflargo.append(fila, ignore_index=True)
    return dflargo.convert_dtypes()

In [ ]:
# Recorta valores poco relevantes. Ritmos con 2 o menos ocurrencias van fuera
def recorta(data, minimo):
    for valor in data.columns:
        if valor != 'Autor' and valor != 'Obra':
            if data[valor].sum() < minimo:
                data = data.drop(columns=valor)
    return data.convert_dtypes()

In [ ]:
# entrenamiento, autor, dflargotest, kn, para_comprobar, testr, alea
def analiza(data, j_test=pd.DataFrame(), test=0.2, alea = 42, plot='rbf', analisis='k',  n=12):
    X = data.drop('Autor', axis=1).values
    y = data['Autor'].values
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test, random_state=alea)
    #y_train = X_train['Autor'].values
    if analisis == 'k':
        from sklearn.neighbors import KNeighborsClassifier
        neighbors = np.arange(1,12)
        train_accuracy =np.empty(len(neighbors))
        test_accuracy = np.empty(len(neighbors))
    
        for i,k in enumerate(neighbors):
            #Setup a knn classifier with k neighbors
            modelo = KNeighborsClassifier(n_neighbors=k)
            #Fit the model
            modelo.fit(X_train,y_train)
    
            #Compute accuracy on the training set
            train_accuracy[i] = modelo.score(X_train,y_train)
    
            #Compute accuracy on the test set
            test_accuracy[i] = modelo.score(X_test, y_test)
        if plot == 'y':
            # Gráfico para calcular la precisión. Según sus resultados, ajustamos
            # el número de vecinos a un valor óptimo
            plt.rcParams["figure.figsize"] = (10,8)
            plt.title('k-NN número de vecinos')
            plt.plot(neighbors, test_accuracy, label='Precisión de la prueba')
            plt.plot(neighbors, train_accuracy, label='Precisión del entrenamiento')
            plt.legend()
            plt.xlabel('Número de vecinos')
            plt.ylabel('Precisión')
            plt.show()
        else:
            print(test_accuracy)
        modelo = KNeighborsClassifier(n_neighbors=n)
        modelo.fit(X_train,y_train)
    elif analisis == 'r':
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
    
        j_test = sc.transform(j_test)
        from sklearn.linear_model import LogisticRegression
        modelo = LogisticRegression(max_iter=1000,  random_state=alea)
        modelo.fit(X_train,y_train)
        y_pred = modelo.predict(X_test)
    elif analisis == 's':
        from sklearn import svm
        modelo = svm.SVC(kernel=plot)
        modelo.fit(X_train,y_train)
    else:
        return False

    #j_testX = sc.transform(j_testX)

    from sklearn.metrics import classification_report
    print(f'Score: {modelo.score(X_test,y_test)}')
    from sklearn.metrics import confusion_matrix
    y_pred = modelo.predict(X_test)
    # Lo mismo que el gráfico
    #print(confusion_matrix(y_test,y_pred))
    # Ditto
    #print(classification_report(y_test,y_pred))
    print(modelo.predict(j_test))
    #print(f'Prueba: {knn.predict(prueba)}')
    return modelo

In [ ]:
def borra(data, columna, ritmosx):
    #data = data.loc[data['Cuenta'] > 10]
    obras = pd.unique(data['Obra'].values)
    ritmosa = pd.unique(data[columna].values)
    for ritmo in ritmosa:
        if ritmo not in ritmosx:
            data = data.loc[a[columna] != ritmo]
        else:
            for autor in ['Calderón', 'Mira', 'Lope']:
                if ritmo not in data.loc[data['Autor'] == autor][columna].values:
                    data = data.loc[data[columna] != ritmo]
            if len(data[data[columna] == ritmo]) < 20 or data.loc[data[columna] == ritmo]['RelCuenta'].max() < 0.05:
                data = data.loc[data[columna] != ritmo]
    return data

In [ ]:
def borrar(data, columna, lista = []):
    data = data.loc[~data['Autor'].isin(control)]
    data = data.loc[~data['Obra'].str.startswith('z', na=False)]
    #data = data.loc[data['Cuenta'] > 10]
    obras = pd.unique(data['Obra'].values)
    x = data.loc[data['Autor'].isin(dudosos)]
    a = data.loc[~data['Autor'].isin(dudosos)]
    if len(lista) == 0:
        ritmosx = pd.unique(x[columna].values)
        ritmosa = pd.unique(a[columna].values)
    else:
        return data.loc[data[columna].isin(lista)]
    for ritmo in ritmosa:
        if ritmo not in ritmosx:
            a = a.loc[a[columna] != ritmo]
        else:
            for autor in ['Calderón', 'Mira', 'Lope']:
                if ritmo not in a.loc[a['Autor'] == autor][columna].values:
                    a = a.loc[a[columna] != ritmo]
            if len(a[a[columna] == ritmo]) < 4 or a.loc[a[columna] == ritmo]['Cuenta'].max() < 4:
                a = a.loc[a[columna] != ritmo]
    dfritmo = pd.DataFrame()
    valores = pd.unique(data[columna])
    obras = pd.unique(data['Obra'])
    for obra in obras:
        cuentas = {}
        autor =  data[data['Obra'] == obra]['Autor'].value_counts()[:1].index.tolist()[0]
        for valor in valores:
            fila = {'Obra': obra, 'Autor': autor}
            cuenta = len(data[data['Obra'] == obra][data[columna] == valor])
            fila = {'Obra': obra, 'Autor': autor, columna: valor, 'Cuenta': cuenta, 'RelCuenta' : cuenta/len(obra)}
            if cuenta > 0:           
                dfritmo = dfritmo.append(fila, ignore_index=True)
    return dfritmo.convert_dtypes()

In [ ]:
dfrecuenta = recuento(df, 'Romance')

In [ ]:
dfcuenta = recuento(df, 'Ritmo')

In [ ]:
dfrplot = dfrecuenta.loc[dfrecuenta['Autor'].isin(candidatos)]
dfplot = dfcuenta.loc[dfcuenta['Autor'].isin(candidatos)]

In [ ]:
ritmosx = df.loc[df['Autor'] == 'Jasón']

ritmosx.groupby('Ritmo').count()
rx = pd.unique(ritmosx['Ritmo'])

In [ ]:
ritmosy = df.loc[df['Autor'] == 'Redención']
ritmosy.groupby('Ritmo').count()
ry = pd.unique(ritmosy['Ritmo'])

In [ ]:
ritmosxy = pd.unique(df.loc[df['Autor'].isin(['Jasón', 'Redención'])]['Romance'])

In [ ]:
#creado = borra(dfplot, 'Ritmo', ritmosx)
#creador = borrar(dfrplot.loc[dfrplot['Autor'].isin(candidatos)], 'Romance', rx)

In [ ]:
creado = dfcuenta.loc[dfcuenta['Autor'].isin(candidatos)]
creado = creado.loc[creado['Ritmo'].isin(rx)]


In [ ]:
cre = borra(creado, 'Ritmo',rx)

In [ ]:
dfrplot = dfrplot.loc[dfrplot['Autor'].isin(candidatos)]

In [ ]:
my_pal = {"Calderón": "r", "Lope": "black", "Mira":"orange"}

In [ ]:
# Ritmo
plt.figure(figsize=(16,8))
plot = sns.boxplot(x='Ritmo', y='RelCuenta', hue='Autor', data=cre, palette=my_pal, medianprops=dict(color="white", alpha=0.7))
plt.xticks(rotation=45)

In [ ]:
# Ritmo
#plt.figure(figsize=(16,8))
#plot = sns.boxplot(x='Romance', y='Cuenta', hue='Autor', data=creador)
#plt.xticks(rotation=45)

In [ ]:
#df.groupby('Obra').size()

In [ ]:
#muestrea = muestrasiguales(df)

In [ ]:
#nmuestras = muestrea[1]
#dfreg = muestrea[0]
#dfregcontrol = dfcontrol
# No son necesarios porque Jasón y La soberbia tienen más versos que algunos del entrenamiento
#dfXreg =  muestrasiguales(
#    df[df['Autor'].isin(['Calderón', 'Lope', 'Mira', 'X'])])
#dfYreg =  muestrasiguales(df[df['Autor'].isin(['Calderón', 'Lope', 'Mira', 'Y'])])

#dftestreg = muestrasiguales(df[df['Autor'].isin(['X','Y'])])


In [ ]:
#dfregcontrol = muestreacontrol[0]
#nmuestrascontrol = muestreacontrol[1]

In [ ]:
dfrcuenta = recuento(df, 'Romance')
dfrlargo = formatolargo('Romance', dfrcuenta)
dfrlargo.to_csv('romances.csv', mode='w', header=True, index=False) 


In [ ]:
dflargo = formatolargo('Ritmo', dfcuenta, 'RelCuenta')

In [ ]:
dflargo = recorta(dflargo, 1)
#dflargodos = recorta(dflargo, 3)
#dflargotres = recorta(dflargo, 3)

In [ ]:
#dflargoseis = recorta(dflargo, 6)
#dflargodiez = recorta dflargo(dflargo, 10)
#dflargodoce = recorta(dflargo, 12)

In [ ]:
entrenamiento = dflargo.loc[dflargo['Autor'].isin(candidatos)].drop('Obra', axis=1)
entrenamiento = entrenamiento.loc[~entrenamiento['Autor'].isin(dudosos)]
#entrenamiento = entrenamiento.loc[~entrenamiento['Obra'].str.startswith('z', na=False)].drop('Obra', axis=1)
dfcontrol =  dflargo.loc[dflargo['Autor'].isin(control)].drop('Obra', axis=1)


In [ ]:
control = ['C01', 'C02', 'C03', 'C04',
           'M01', 'M02', 'M03', 'M04']

In [ ]:
dflargo

In [ ]:
# Ajusto los parámetros
testr = 0.01
testk = 0.2
tests = 0.01
#0.16
alear = 42
aleak = 42
aleas = 42
kn = 8
data = dflargo
autores = control
for autor in  ['Redención', 'Jasón']:
#def analiza(tabla, j_test=pd.DataFrame(), test=0.2, alea = 42, plot='n', analisis='k',  n=12):
    print(f'\n\n\n******************************************\n\nObra:\t{autor}: {dflargo.loc[dflargo["Autor"] == autor]["Obra"].max()}\n')
    para_comprobar = dflargo.loc[dflargo['Autor'] == autor].drop('Autor', axis=1).drop('Obra', axis=1).values
    
    print('Regresión')
    analiza(entrenamiento, para_comprobar, testr, alear,'n', 'r')
  
    print('\n-------------------------------------------------------\n\nSVN')
    #testY = dfreglargo[dfreglargo['Autor'] == autor].drop('Autor', axis=1).values
    analiza(entrenamiento, para_comprobar, tests, aleas, 'rbf', 's')
    
    print('\n-------------------------------------------------------\n\nKnn:\n')
    analiza(entrenamiento, para_comprobar, testk, aleak, 'y', 'k', kn)

In [ ]:
#dfromancesY.groupby(['Romance', 'Cuenta']).count()
# No hay ritmos sin instancia

In [ ]:
def creaformula(columna, data):
    variables = []
    lista = pd.unique(data[columna])
    for idx, elemento in enumerate(lista):
        if idx == 0:
            formula = f'{elemento} ~ Autor'
        else:
            formula = f'{elemento} + {formula}'
        variables += [elemento]
    return formula
    

In [ ]:
formularitmos = creaformula('Ritmo', dfritmosCLM)
formularomances = creaformula('Romance', dfromancesX)

In [ ]:
#for idx, i in enumerate(pd.unique(dfritmosCLM['Ritmo'])):
    #print(f'anova{idx} <- MANOVA.wide({i}  ~ Autor, data = ridf, iter = 100, resampling = "WildBS", seed=123,CPU = 4)')
    #print(i)

In [ ]:

entrenamientoX = dfritmolargoX[dfritmolargoX['Autor'] != 'X'][['Autor'] + variablesX]

In [ ]:
cadena = 'p.adjust(c('
for idx, i in enumerate(pd.unique(dfritmosX['Ritmo'])):
    cadena += f'anova{idx}$resampling[, 2],'
cadena += '), method = "bonferroni")'

In [ ]:
from statsmodels.multivariate.manova import MANOVA
maov = MANOVA.from_formula(formularomances, data = dfromancelargo)

In [ ]:
print(maov.mv_test())

In [ ]:
from statsmodels.multivariate.manova import MANOVA
maov = MANOVA.from_formula(formularomances, data = dfromancelargo)

In [ ]:
print(maov.mv_test())

In [ ]:
#clasifica = logreg(entrenamientoX, 'X', testX)
#clasifica = logreg(entrenamientoY, 'Y', testY)

In [ ]:
#testX

In [ ]:
#clasifica = vecinos(entrenamientoX, 'X', 5, testX)

In [ ]:
#clasifica = vecinos(dfritmolargoY, 'Y', 5)

In [ ]:
clasifica = logreg(dfritmolargoX, 'X')

In [ ]:
clasifica = logreg(dfritmolargoY, 'Y')

In [ ]:
logmodelY = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=99)
predictionsY = logmodelY.predict(X_testY)

In [ ]:
#from sklearn.linear_model import LogisticRegression
#scikit_no_penalty = LogisticRegression(random_state=0,penalty='none').fit(X_train, y_train)
#print(f"intecept: {scikit_no_penalty.intercept_} coeficients: {scikit_no_penalty.coef_}")
#print(f"train accuracy: {scikit_no_penalty.score(X_train, y_train)}")
#print(f"test accuracy: {scikit_no_penalty.score(X_test, y_test)}")

In [ ]:
#from sklearn.linear_model import LogisticRegression
#scikit_no_penalty = LogisticRegression(random_state=0,penalty='none').fit(X_train, y_train)
#print(f"intecept: {scikit_no_penalty.intercept_} coeficients: {scikit_no_penalty.coef_}")
#print(f"train accuracy: {scikit_no_penalty.score(X_train, y_train)}")
#print(f"test accuracy: {scikit_no_penalty.score(X_test, y_test)}")


In [ ]:
#from sklearn.metrics import roc_auc_score
#ascore = roc_auc_score(y_testX, logreg_model.predict_proba(X_testX)[:,1])

In [ ]:
#ascore

In [ ]:
#knnX.score(X_testX, y_testX)logreg_model.summary()

In [ ]:
#result = pd.concat([df, result_df], axis=1)

In [ ]:
#result

In [ ]:
#log_reg = sm.Logit(np.asarray(entrenamiento['Autor']), np.asarray(entrenamiento[variables])).fit()

In [ ]:
#log_reg = sm.Logit(y_train, X_train).fit()

In [ ]:
#log_reg.fit()

In [ ]:
#from patsy import dmatrices

In [ ]:
#y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
boxplot = df.boxplot(column=['eo'], by=['Autor'])

test

In [ ]:
boxplot = df.boxplot(column=aso, by=['Autor'])

In [ ]:
boxplot = dtest.boxplot(column=vocales, by=['Autor'])

In [ ]:
todos = pd.concat([ddf, dtest])

In [ ]:

df.to_csv(f'/tmp/final.csv', mode='w', header=True, index=False)
test.to_csv(f'/tmp/final_test.csv', mode='w', header=True, index=False)
